# Problem Set 4 -Yutao Chen
## Estimating the Brock and Mirman (1972) model by SMM

In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import matplotlib
import itertools
import seaborn as sns
import math
import sympy as sy
import warnings
import statsmodels.tsa.ar_model as ar
warnings.filterwarnings("ignore")

F:\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:49: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)


In [11]:
# data loading
macro_data = np.loadtxt('data/NewMacroSeries.txt', dtype=float, delimiter=',')
macro_data.shape

(100, 5)

### (a) Estimate four parameters $(\alpha, \rho, \mu, \sigma)$ given $\beta = 0.99$ of the Brock and Mirman (1972) model described by equations (1) through (6) by SMM...

In [3]:
# data simulation
def data_simulation(paras, basis_draw, initials, S=1000, T=100):
    alpha, rho, mu, sigma = paras
    beta = 0.99
    z0, k1 = initials
    # Inverse transform sampling
    error_term = sp.stats.norm.ppf(basis_draw, scale=sigma)
    
    # construct simulated zt, kt
    zt = np.zeros([T,S])
    zt[0,:] = z0 + error_term[0,:]
    kt = np.zeros([T,S])
    kt[0,:] = k1
    for i in range(1,T):
        zt[i,:] = zt[i-1,:]*rho + (1-rho)*mu + error_term[i,:]
        kt[i,:] = alpha*beta*np.exp(zt[i-1,:])*(kt[i-1,:])**alpha
    
    # construct simulated wt, rt, yt ,ct
    wt = np.zeros([T,S])
    rt = np.zeros([T,S])
    yt = np.zeros([T,S])
    ct = np.zeros([T,S])
    for i in range(T):
        wt[i,:] = (1-alpha)*np.exp(zt[i,:])*(kt[i,:])**alpha
        rt[i,:] = alpha*np.exp(zt[i,:])*(kt[i,:])**(alpha-1)
        yt[i,:] = np.exp(zt[i,:])*(kt[i,:])**alpha
        ct[i,:] = rt[i,:]*kt[i,:]+wt[i,:] - (alpha*beta*np.exp(zt[i,:])*(kt[i,:])**alpha) #k_(t+1)

    return (zt,ct,kt,wt,rt,yt)

In [4]:
# construct moments
def data_moments(data):
    ct = data[:,0]
    kt = data[:,1]
    wt = data[:,2]
    rt = data[:,3]
    yt = data[:,4]
    m1 = np.mean(ct)
    m2 = np.mean(kt)
    m3 = np.mean(ct/yt)
    m4 = np.var(yt)
    m5 = np.corrcoef(ct[:-1],ct[1:])[0,1]
    m6 = np.corrcoef(ct,kt)[0,1]
    return np.array((m1,m2,m3,m4,m5,m6))

def simulated_moments(simulation):
    zt,ct,kt,wt,rt,yt = simulation
    S = zt.shape[1]
    m1 = np.mean(ct,axis=0)
    m2 = np.mean(kt,axis=0)
    m3 = np.mean(ct/yt, axis=0)
    m4 = np.var(yt, axis=0)
    m5 = [np.corrcoef(ct[:-1,i],ct[1:,i])[0,1] for i in range(S)]
    m6 = [np.corrcoef(ct[:,i],kt[:,i])[0,1] for i in range(S)]
    return np.array((m1,m2,m3,m4,m5,m6))

In [5]:
# error vectors and criterion function
def err_vec(data, simulation, simple):
    Data_Moments = data_moments(data)
    Simulated_Moments = simulated_moments(simulation)
    Ave_Simulated_Moments = np.mean(Simulated_Moments,axis=1)
    if simple:
        return Ave_Simulated_Moments-Data_Moments
    else:
        return (Ave_Simulated_Moments-Data_Moments)/Data_Moments
    
def criterion(paras, *args):
    alpha, rho, mu, sigma = paras
    data, basis_draw, simple, W = args
    k1 = np.mean(data[:,1])
    simulated_data = data_simulation(paras, basis_draw, (mu,k1))
    error = err_vec(data, simulated_data, simple)
    return error.T @ W @ error

In [6]:
# initialize
def zt4(alpha, data):
    rt = data[:,3]
    kt = data[:,1]
    return (np.log(rt)-np.log(alpha)-(alpha-1)*np.log(kt))

def AR1(zt):
    AR = ar.AR(zt)
    AR1_result = AR.fit(maxlag=1,trend='c')
    rho = AR1_result.params[1]
    mu = AR1_result.params[0]/(1-rho)
    sigma = (AR1_result.sigma2)**0.5
    return rho,mu,sigma

In [7]:
# optimization 
# a matrix of S = 1000 simulations (columns) of T = 100 (rows) from a uniform us,t ∼ U(0, 1)
basis_draw = np.random.uniform(0,1,(100,1000))
macro_result = []
W_identity = np.eye(6)
for i in np.arange(0.01,0.99,0.1):
    alpha_init = i
    z_init = zt4(alpha_init, macro_data)    
    rho_init, mu_init, sigma_init = AR1(z_init)
    temp_result = sp.optimize.minimize(criterion, np.array((alpha_init, rho_init, mu_init, sigma_init)), 
                                       args = (macro_data, basis_draw, False, W_identity), 
                                       options={'maxiter':2000}, method="SLSQP",
                                       bounds=[(0.01, 0.99),(-0.99, 0.99),(5, 14),(0.01, 1.1)])
    macro_result.append(temp_result)

In [8]:
identity_result = sorted(macro_result, key=lambda x: x.fun)[0]
identity_result

     fun: 4.325180126681273e-06
     jac: array([0.03355759, 0.00581554, 0.00190708, 0.01324586])
 message: 'Optimization terminated successfully.'
    nfev: 104
     nit: 16
    njev: 16
  status: 0
 success: True
       x: array([0.42102413, 0.92214702, 9.93513811, 0.08844615])

In [11]:
# report estimations and criterion function value
print('criterion function value: {}'.format(identity_result.fun))
print('estimated alpha: {}'.format(identity_result.x[0]))
print('estimated rho: {}'.format(identity_result.x[1]))
print('estimated mu: {}'.format(identity_result.x[2]))
print('estimated sigma: {}'.format(identity_result.x[3]))

criterion function value: 4.325180126681273e-06
estimated alpha: 0.4210241261677023
estimated rho: 0.9221470215474382
estimated mu: 9.935138113603411
estimated sigma: 0.08844614583724006


In [19]:
# report error vector  
z0 = identity_result.x[2]
k1 = np.mean(macro_data[:,1])
identity_simulation = data_simulation(identity_result.x, basis_draw, (z0,k1))
print(err_vec(macro_data, identity_simulation, False))

[ 0.00077862 -0.00070536 -0.00173551  0.00023562  0.00028136 -0.00027336]


In [17]:
# calculate std matrix for parameter estimators using identity weighting matrix
def Jac_matrix(data, basis_draw, paras):
    Jac_matrix = np.zeros((6, len(paras)))
    h = 1e-8
    for i in range(len(paras)):
        temp_paras_up = list(paras)
        temp_paras_up[i] *= (1+h)
        up_simulation = data_simulation(temp_paras_up, basis_draw, (temp_paras_up[2] ,np.mean(data[:,1])))
        temp_paras_down = list(paras)
        temp_paras_down[i] *= (1-h)
        down_simulation = data_simulation(temp_paras_down, basis_draw, (temp_paras_down[2] ,np.mean(data[:,1])))
        Jac_matrix[:,i] = (err_vec(data, up_simulation, False) - err_vec(data, down_simulation, False))/(2*h*paras[i])
    return Jac_matrix

identity_d_err = Jac_matrix(macro_data, basis_draw, tuple(para for para in identity_result.x))
identity_std_matrix = (1/1000) * np.linalg.inv(identity_d_err.T @ W_identity @ identity_d_err)
print(identity_std_matrix)

[[ 9.02311178e-05 -2.70246693e-05 -1.51777851e-03  1.41221550e-06]
 [-2.70246693e-05  2.35024522e-03  2.54801233e-04 -9.99178799e-04]
 [-1.51777851e-03  2.54801233e-04  2.57630537e-02  3.32075793e-05]
 [ 1.41221550e-06 -9.99178799e-04  3.32075793e-05  4.30923700e-04]]


In [39]:
print("std err for estimated alpha: {}".format(np.sqrt(identity_std_matrix[0,0])))
print("std err for estimated rho: {}".format(np.sqrt(identity_std_matrix[1,1])))
print("std err for estimated mu: {}".format(np.sqrt(identity_std_matrix[2,2])))
print("std err for estimated sigma: {}".format(np.sqrt(identity_std_matrix[3,3])))

std err for estimated alpha: 0.009499006149067369
std err for estimated rho: 0.04847932777731694
std err for estimated mu: 0.16050873403502097
std err for estimated sigma: 0.02075870178749786


### (b) Perform the estimation using the two-step estimator for the optimal weighting matrix...

In [30]:
# construct the error matrix
def err_matrix(paras, data, basis_draw, simple):
    z0 = paras[2]
    k1 = np.mean(data[:,1])
    Simulated_Data = data_simulation(paras, basis_draw, (z0,k1))
    Data_Moments = data_moments(data)
    Simulated_Moments = simulated_moments(Simulated_Data)
    error_matrix = []
    for i in range(len(Data_Moments)):
        if simple:
            error_matrix.append(Simulated_Moments[i]-Data_Moments[i]) 
        else:
            error_matrix.append((Simulated_Moments[i]-Data_Moments[i])/Data_Moments[i])
    return np.array(error_matrix)

# calculate the two-step optimal weighting matrix
macro_err_matrix = err_matrix(tuple(para for para in identity_result.x), macro_data, basis_draw, False)
macro_VCV = (1/macro_err_matrix.shape[1])*(macro_err_matrix @ macro_err_matrix.T)
W_2step = np.linalg.inv(macro_VCV)
W_2step

array([[ 6.58576887e+04, -6.64410000e+04,  5.36371926e+04,
         3.69468450e+01, -9.59232285e+03,  8.64990957e+03],
       [-6.64410000e+04,  6.70842949e+04, -5.40096555e+04,
        -4.68313522e+01,  1.01259819e+04, -9.06581578e+03],
       [ 5.36371926e+04, -5.40096555e+04,  4.41833950e+05,
         3.60816536e+01,  1.99395665e+05, -1.99871165e+05],
       [ 3.69468450e+01, -4.68313522e+01,  3.60816536e+01,
         3.88787374e+00, -2.77924314e+01, -2.82533592e+01],
       [-9.59232285e+03,  1.01259819e+04,  1.99395665e+05,
        -2.77924314e+01,  6.51288680e+05, -6.49033235e+05],
       [ 8.64990957e+03, -9.06581578e+03, -1.99871165e+05,
        -2.82533592e+01, -6.49033235e+05,  6.48905391e+05]])

In [33]:
# two step estimation
twostep_result = sp.optimize.minimize(criterion, np.array([para for para in identity_result.x]), 
                                      args = (macro_data, basis_draw, False, W_2step), 
                                      options={'maxiter':2000}, method="SLSQP",
                                      bounds=[(0.01, 0.99),(-0.99, 0.99),(5, 14),(0.01, 1.1)])
twostep_result

     fun: 0.6855048859394466
     jac: array([ 0.17131718, -0.01272216, -0.0121364 , -0.07730648])
 message: 'Optimization terminated successfully.'
    nfev: 54
     nit: 7
    njev: 7
  status: 0
 success: True
       x: array([0.42070079, 0.92353363, 9.93460106, 0.08721862])

In [34]:
# report estimations and criterion function value
print('criterion function value: {}'.format(twostep_result.fun))
print('estimated alpha: {}'.format(twostep_result.x[0]))
print('estimated rho: {}'.format(twostep_result.x[1]))
print('estimated mu: {}'.format(twostep_result.x[2]))
print('estimated sigma: {}'.format(twostep_result.x[3]))

criterion function value: 0.6855048859394466
estimated alpha: 0.4207007863198329
estimated rho: 0.9235336260125493
estimated mu: 9.93460106040918
estimated sigma: 0.08721862232911898


In [35]:
# report error vector  
z0 = twostep_result.x[2]
k1 = np.mean(macro_data[:,1])
twostep_simulation = data_simulation(twostep_result.x, basis_draw, (z0,k1))
print(err_vec(macro_data, twostep_simulation, False))

[-0.00954838 -0.01220246 -0.00118757 -0.0379656   0.000929    0.000351  ]


In [38]:
# calculate std matrix for twostep parameter estimators
twostep_d_err = Jac_matrix(macro_data, basis_draw, tuple(para for para in twostep_result.x))
twostep_std_matrix = (1/1000) * np.linalg.inv(twostep_d_err.T @ W_2step @ twostep_d_err)
print(twostep_std_matrix)

[[ 3.32393725e-10 -1.46399408e-09  4.08686111e-10  1.27883401e-09]
 [-1.46399408e-09  3.90267714e-06 -7.43519672e-07 -5.43892273e-07]
 [ 4.08686111e-10 -7.43519672e-07  9.67996704e-06  5.41598558e-08]
 [ 1.27883401e-09 -5.43892273e-07  5.41598558e-08  4.88013228e-07]]


In [40]:
print("std err for estimated alpha: {}".format(np.sqrt(twostep_std_matrix[0,0])))
print("std err for estimated rho: {}".format(np.sqrt(twostep_std_matrix[1,1])))
print("std err for estimated mu: {}".format(np.sqrt(twostep_std_matrix[2,2])))
print("std err for estimated sigma: {}".format(np.sqrt(twostep_std_matrix[3,3])))

std err for estimated alpha: 1.8231668188248857e-05
std err for estimated rho: 0.0019755194607543494
std err for estimated mu: 0.003111264540347709
std err for estimated sigma: 0.0006985794355372651
